# Llama Debug Handler Demo

Here we showcase the capabilities of our LlamaDebugHandler in logging events as we run queries
within LlamaIndex.

**NOTE**: This is a beta feature. The usage within different classes and the API interface
    for the CallbackManager and LlamaDebugHandler may change!

In [1]:
from llama_index.callbacks import CallbackManager, LlamaDebugHandler, CBEventType

/usr/local/anaconda3/envs/langchain/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from llama_index import GPTListIndex, ServiceContext, SimpleDirectoryReader

In [4]:
docs = SimpleDirectoryReader("../paul_graham_essay/data").load_data()

## Callback Manager Setup

In [6]:
llama_debug = LlamaDebugHandler()
callback_manager = CallbackManager([llama_debug])
service_context = ServiceContext.from_defaults(callback_manager=callback_manager)

## Trigger the callback with a query

In [7]:
index = GPTListIndex.from_documents(docs, service_context=service_context)
query_engine = index.as_query_engine()

INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 0 tokens


In [8]:
response = query_engine.query("What did the author do growing up?")

INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 28944 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens


## Explore the Debug Information

The callback manager will log several start and end events for the following types:
- CBEventType.LLM
- CBEventyType.EMBEDDING
- CBEventType.CHUNKING
- CBEventType.RETRIEVE
- CBEventType.SYNTHESIZE 
- CBEventType.TREE
- CBEventType.QUERY

The LlamaDebugHandler provides a few basic methods for exploring information about these events

In [9]:
# Print info on the LLM calls during the list index query
print(llama_debug.get_event_time_info(CBEventType.LLM))

EventStats(total_secs=161.0, average_secs=13.416666666666666, total_count=12)


In [18]:
# Print info on llm inputs/outputs - returns start/end events for each LLM call
event_pairs = llama_debug.get_llm_inputs_outputs()
print(event_pairs[0][0])
print(event_pairs[0][1].payload.keys())
print(event_pairs[0][1].payload['response'])

CBEvent(event_type=<CBEventType.LLM: 'llm'>, payload=None, time='05/01/2023, 19:58:56', id_='1fd2e2c5-a334-47cc-9f81-188035825633')
dict_keys(['stage', 'response', 'formatted_prompt'])

Growing up, the author wrote short stories, programmed on an IBM 1401, and wrote programs for a TRS-80 microcomputer. He wrote simple games, a program to predict how high his model rockets would fly, and a word processor. He also studied philosophy in college, but switched to AI when he realized it was a more exciting field. He reverse-engineered SHRDLU for his undergraduate thesis, and wrote a book about Lisp hacking while in grad school. He also took art classes at Harvard and applied to art schools.


In [23]:
# Get info on any event type
event_pairs = llama_debug.get_event_pairs(CBEventType.CHUNKING)
print(event_pairs[0][0].payload.keys())  # get first chunking start event
print(event_pairs[0][1].payload.keys())  # get first chunking end event

dict_keys(['documents'])
dict_keys(['nodes'])


In [25]:
# Clear the currently cached events
llama_debug.flush_event_logs()